In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchsummary

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train_dir = '/opt/ml/input/data/train'

data_info = pd.read_csv(train_dir + '/train.csv', header=0)

In [3]:
data_info.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [4]:
file_names = ['/incorrect_mask.jpg', '/mask1.jpg','/mask2.jpg','/mask3.jpg','/mask4.jpg','/mask5.jpg', '/normal.jpg']

prepro_data_info = pd.DataFrame(columns={'id','img_path','race','mask','gender','age','label'})

all_id, all_path, all_race, all_mask, all_age, all_gender, all_label = [],[],[],[],[],[],[]

In [5]:
for absolute_path in glob(train_dir + "/images/*/*"):
    split_list = absolute_path.split("/")
    img_name = split_list[-1]
    img_path = split_list[-2]
    
    path_split = img_path.split("_")
    
    img_id = path_split[0]
    img_gender = 0 if path_split[1] == "male" else 1
    img_race = path_split[2]
    img_age = min(2, int(path_split[3]) // 30)
    
    img_mask = 0
    if 'incorrect' in img_name:
        img_mask = 1
    elif 'normal' in img_name:
        img_mask = 2
    
    all_id.append(img_id)
    all_path.append(absolute_path)
    all_race.append(img_race)
    all_mask.append(img_mask)
    all_gender.append(img_gender)
    all_age.append(img_age)
    all_label.append(img_mask*6 + img_gender*3 + img_age)


In [6]:
prepro_data_info['id'] = all_id
prepro_data_info['img_path'] = all_path
prepro_data_info['race'] = all_race
prepro_data_info['mask'] = all_mask
prepro_data_info['gender'] = all_gender
prepro_data_info['age'] = all_age
prepro_data_info['label'] = all_label

In [7]:
prepro_data_info.head()

,gender,label,race,age,id,mask,img_path
0,1,3,Asian,0,003124,0,/opt/ml/input/data/train/images/003124_female_...
1,1,9,Asian,0,003124,1,/opt/ml/input/data/train/images/003124_female_...
2,1,15,Asian,0,003124,2,/opt/ml/input/data/train/images/003124_female_...
3,1,3,Asian,0,003124,0,/opt/ml/input/data/train/images/003124_female_...
4,1,3,Asian,0,003124,0,/opt/ml/input/data/train/images/003124_female_...


In [8]:
class MyCustomDataset(Dataset):
    def __init__(self, prepro_data_info, train=True):
        self.data_info = prepro_data_info # preprocessFunction()

        self.to_tensor = transforms.ToTensor()

        self.image_path = self.data_info.img_path.tolist()
        self.label_arr = self.data_info.label.tolist()

        self.data_len = len(self.data_info.img_path)
        
        self.train = train

    def __getitem__(self, index):
        img_thing = Image.open(self.image_path[index])
        
        img_thing = img_thing.resize((24, 32))
        img_thing = np.asarray(img_thing)/255
        
        img_tensor = self.to_tensor(img_thing)
        
        if self.train:
            img_label = self.label_arr[index]
            return (img_tensor, img_label)
        else:
            return img_tensor

    def __len__(self):
        return self.data_len

In [9]:
LR = 0.01
SEED = 42
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_data, valid_data = train_test_split(prepro_data_info, test_size=0.20, random_state=SEED)

print(train_data.shape, valid_data.shape)

(15120, 7) (3780, 7)


In [10]:
my_custom_dataset = MyCustomDataset(train_data)

In [11]:
img, _ = my_custom_dataset[1]

In [12]:
img.shape

torch.Size([3, 32, 24])

In [13]:
len(my_custom_dataset)

15120

In [14]:
class MyCustomModel(nn.Module):
    
    def __init__(self):
        super(MyCustomModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(16*14*10, 18)
        
    def forward(self, x):
        x = self.cnn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
        

In [15]:
my_model = MyCustomModel()
my_model.to(device)
print(my_model)

MyCustomModel(
  (cnn1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2240, out_features=18, bias=True)
)


In [16]:
torchsummary.summary(my_model, (3, 32, 24), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 20]           1,216
              ReLU-2           [-1, 16, 28, 20]               0
         MaxPool2d-3           [-1, 16, 14, 10]               0
            Linear-4                   [-1, 18]          40,338
Total params: 41,554
Trainable params: 41,554
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.15
Params size (MB): 0.16
Estimated Total Size (MB): 0.32
----------------------------------------------------------------


In [17]:
from torch.autograd import Variable

my_dataset_loader = torch.utils.data.DataLoader(dataset=my_custom_dataset, batch_size=14, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_model.parameters(), lr=LR)

for epoch in range(2):
    for i, (images, labels) in enumerate(my_dataset_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = my_model(images.float())
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        if i%200 == 0:
            print(f"Eopch: {epoch} | Batch ID: {i} | Loss: {loss.data}")

print("done!")

Eopch: 0 | Batch ID: 0 | Loss: 2.8517744541168213
Eopch: 0 | Batch ID: 200 | Loss: 2.2389233112335205
Eopch: 0 | Batch ID: 400 | Loss: 1.9165019989013672
Eopch: 0 | Batch ID: 600 | Loss: 1.8566482067108154
Eopch: 0 | Batch ID: 800 | Loss: 1.7792977094650269
Eopch: 0 | Batch ID: 1000 | Loss: 1.6436325311660767
Eopch: 1 | Batch ID: 0 | Loss: 1.656982183456421
Eopch: 1 | Batch ID: 200 | Loss: 1.4331414699554443
Eopch: 1 | Batch ID: 400 | Loss: 1.4985780715942383
Eopch: 1 | Batch ID: 600 | Loss: 1.1567212343215942
Eopch: 1 | Batch ID: 800 | Loss: 1.1487308740615845
Eopch: 1 | Batch ID: 1000 | Loss: 1.226937174797058
done!


In [18]:
valid_dataset = MyCustomDataset(valid_data)

valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=14, shuffle=False)

In [19]:
targets = []
all_predictions = []

for images, labels in valid_dataloader:
    with torch.no_grad():
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        pred = my_model(images.float())
        pred = pred.argmax(dim=-1)
        
        targets.extend(labels.cpu().numpy())
        
        all_predictions.extend(pred.cpu().numpy())

print(f"Accuracy: {accuracy_score(targets, all_predictions)}")
print(f"F1 Score: {np.mean(f1_score(targets, all_predictions, average=None))}")

Accuracy: 0.5857142857142857
F1 Score: 0.3100556696333911
